In [24]:
import pandas as pd
from collections import Counter
from sklearn.mixture import GaussianMixture
import numpy as np

In [2]:
%pwd

'/opt/notebooks/CS 6220/Assignments'

## Fashion MNIST

In [3]:
fashion_train_df = pd.read_csv('./data/fashionmnist/fashion-mnist_train.csv')
fashion_test_df = pd.read_csv('./data/fashionmnist/fashion-mnist_test.csv')

In [5]:
X_train = fashion_train_df.iloc[:,fashion_train_df.columns != 'label']
X_test = fashion_test_df.iloc[:,fashion_test_df.columns != 'label']
Y_train = fashion_train_df[['label']]
Y_test =  fashion_test_df[['label']]

In [9]:
estimatordiag = GaussianMixture(n_components=10, covariance_type='diag')

In [10]:
estimatordiag.fit(X_train)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [11]:
estimatordiag.covariances_.shape

(10, 784)

In [13]:
estimator.fit(X_train)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [14]:
yhat_test = estimator.predict(X_test)
yhat_train = estimator.predict(X_train)

In [15]:
estimator.covariances_.shape

(10, 784, 784)

In [145]:
yhat_test

array([2, 7, 9, ..., 4, 0, 9])

In [16]:
counter = Counter() 
for i in yhat_test: counter[i] += 1

In [17]:
counter.most_common()

[(1, 2017),
 (5, 1907),
 (3, 1871),
 (6, 1728),
 (4, 1108),
 (0, 656),
 (7, 276),
 (9, 221),
 (8, 216)]

In [18]:
purity_df = pd.DataFrame([yhat_test, Y_test['label']]).transpose()

In [19]:
purity_df.columns = ['yhat_test', 'y_test']

In [20]:
purity_df

,yhat_test,y_test
0,9,0
1,6,1
2,3,2
3,8,2
4,6,3
5,7,2
6,1,8
7,5,6
8,4,5
9,3,0


In [21]:
purity_numerator = 0
purity_denominator = len(purity_df)

for cluster in purity_df['yhat_test'].unique():
    
    purity_numerator += int(purity_df[yhat_test == cluster].groupby('y_test')\
                                .count()\
                                .sort_values('yhat_test', ascending=False)\
                                .iloc[0])

In [22]:
purity_df[yhat_test == cluster].groupby('y_test')\
                                .sum()\
                                .sort_values('yhat_test', ascending=False)


,yhat_test
y_test,
0,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [185]:
purity_numerator/purity_denominator

0.4716

In [28]:
gini_coeffs = np.zeros([len(purity_df['yhat_test'].unique()), 2])
i = 0

for cluster in yhat_train:

    if (i < len(set(list(yhat_train)))-1):
        
        gini_coeffs[i][0] = sum(((purity_df[purity_df['yhat_test'] == cluster].groupby('y_test')\
                                    .count()\
                                    .sort_values('yhat_test', ascending=False)\
                            /len(purity_df[purity_df['yhat_test'] == cluster]))\
                            **2)\
                            ['yhat_test'])

        gini_coeffs[i][1] = len(purity_df[purity_df['yhat_test'] == cluster])

        i += 1
    
    else: break

sum(gini_coeffs[:,0] * gini_coeffs[:,1]) / sum(gini_coeffs[:,1])

0.26357712970030056